In [3]:
def preprocessor(df1,date):
    import pandas as pd
    import numpy as np
    from sklearn import preprocessing
    #df1=df1[df1['Status']=='Overdue']
    #print(df1['Status'])
    df1.dropna(inplace=True)
    df1 = df1.reset_index()
    df1.drop(columns='index',inplace=True)
    
    
    
    df1['Pstng.Date']=pd.to_datetime(df1['Pstng.Date'],dayfirst=True).dt.strftime("%Y%m%d").astype(int)
    df1['Net.Due.Dt']=pd.to_datetime(df1['Net.Due.Dt'],dayfirst=True).dt.strftime("%Y%m%d").astype(int)
    df1['Clearing']=pd.to_datetime(df1['Clearing']).dt.strftime("%Y%m%d").astype(int)   
    df=df1[df1['Pstng.Date'] < date ]
    df3=df[df['Clearing'] > date] 
    df.drop(df[df['Clearing'] > date].index,inplace=True)
    df1=df
    df1 = df1.reset_index()
    df1.drop(columns='index',inplace=True)
    
    
    df3 = df3.reset_index()
    df3.drop(columns='index',inplace=True)
    
    
    df2=df1[['Pstng.Date','Net.Due.Dt','Clearing','Status']]
    df2.dropna(inplace=True)
    df2['Range of Delay']=pd.to_datetime(df2['Clearing'],format='%Y%m%d') - pd.to_datetime(df2['Net.Due.Dt'],format='%Y%m%d')
    df2['PayT']= pd.to_datetime(df2['Net.Due.Dt'],format='%Y%m%d') - pd.to_datetime(df2['Pstng.Date'],format='%Y%m%d')
    df2['Range of Delay']=df2['Range of Delay'].dt.days
    df2['Range of Delay']=pd.to_numeric(df2['Range of Delay'])
    df2['PayT']=df2['PayT'].dt.days
    df2['PayT']=pd.to_numeric(df2['PayT'])
    
    df2 = df2.reset_index()
    df2.drop(columns = 'index',inplace=True)
    
    df2['Status'] = np.where(df2['Range of Delay'] > 7, 2, (np.where(df2['Range of Delay'] > 0, 1,0)))#(np.where(df2['Range of Delay'] > 0, 1,0)))))#(np.where(df2['Range of Delay'] > 0, 1,0)))))#(np.where(df2['Range of Delay'] > 10, 2,(np.where(df2['Range of Delay'] > 0,1,0)))))))# (np.where(df2['Range of Delay'] > 30, 5, (np.where(df2['Range of Delay'] > 15, 4, (np.where(df2['Range of Delay'] > 7, 3, (np.where(df2['Range of Delay'] > 3, 2, (np.where(df2['Range of Delay'] > 0, 1 ,0)))))))))))))))
    #print(df2)
    print('Y s in the initial set')
    print(np.unique(df2['Status'],return_counts=True))
    print('----------------------------------------------------------------------------------------')
    
    df1['Status']=df2['Status']
    df1['PayT']=df2['PayT']
    df1.dropna(inplace=True)
    y=df1['Status']
    
    
    busa = preprocessing.LabelEncoder()
    ccar= preprocessing.LabelEncoder()
    month=preprocessing.LabelEncoder()
    zone=preprocessing.LabelEncoder()
    bran=preprocessing.LabelEncoder()
    payt=preprocessing.LabelEncoder()
    
    df1.drop(columns='Status',inplace=True)
    #df1['Status']=df1['Status'].astype('category').cat.codes
    arr=busa.fit(df1['BusA'])
    df1['BusA']=busa.transform(df1['BusA'])
    arr=ccar.fit(df1['CCAr'])
    df1['CCAr']=ccar.transform(df1['CCAr'])#.astype('category').cat.codes
    #df1['Account']=df1['Account'].astype('category').cat.codes
    arr=month.fit(df1['Month'])
    df1['Month']=month.transform(df1['Month'])#.astype('category').cat.codes
    df1.drop(columns='Reference',inplace=True)
    df1.drop(columns='Customer.Name',inplace=True)
    df1.drop(columns=['DocumentNo','Year','Clrng.doc.'],inplace=True)
    arr=zone.fit(df1['Zone'])
    df1['Zone']=zone.transform(df1['Zone'])    #.astype('category').cat.codes
    arr=bran.fit(df1['Bran'])
    df1['Bran']=bran.transform(df1['Bran'])#.astype('category').cat.codes
    arr =payt.fit(df1['PayT'])
    #df1['PayT'] = payt.transform(df1['PayT'])#.astype('category').cat.codes
    df1.drop(columns='Doc.Chq.dt',inplace=True)
    df1.drop(columns='Ty',inplace=True)
    df1.drop(columns='Sale.Type',inplace=True)
    
    
    #df1.drop(columns='Clearing',inplace=True)
    df1 = df1.reset_index()
    df1.drop(columns='index',inplace=True)
    df1.drop(columns='Arr..Clearing...Net.Due.Date.',inplace=True)
    df1.drop(columns='G.L',inplace=True)
    df1.drop(columns='Clearing',inplace=True)
    

    
    
    #print('Creating Red Alert Zone')
    #table = pd.crosstab(columns=y, index=df1['Account'])
    #table.columns= ["No delay"," 0 - 10", ">10"]
    #print(table)
    #print(table.sort_values(['>10'],ascending=False).head())
    #arr=table.sort_values(['>10'],ascending=False)
    #arr.drop(columns=["No delay"," 0 - 10"],inplace=True)
    #df_1=arr[arr['>10']>40]
    #df_2=[]
    
    
    #df1['Account']=df1['Account']/100000
    #df1['Reference']=df1['Reference']/1000000000
    #df1['Pstng.Date']=df1['Pstng.Date']/100000000
    #df1['Net.Due.Dt']=df1['Pstng.Date']/100000000
    


    #print(df_1.index)
    
    
    ################### Oversampling ##############################
    from imblearn.over_sampling import ADASYN 
    sm = ADASYN()
    df4, y = sm.fit_resample(df1, y)
    df1=pd.DataFrame(df4,columns=df1.columns)
    ############################################################
    
    #y1=df1.columns
    #x = df1.values #returns a numpy array
    #min_max_scaler = preprocessing.MinMaxScaler()
    #x_scaled = min_max_scaler.fit_transform(x)
    #df1 = pd.DataFrame(x_scaled,columns=df1.columns)
    
    print('Y s in the set')
    print(np.unique(y,return_counts=True))
    print('----------------------------------------------------------------------------------------')
    
    return df1,df3,y,busa,ccar,month,zone,bran,payt

In [4]:
def trainer(df1,y):
    import pandas as pd
    import numpy as np
    from sklearn import metrics
    from sklearn.ensemble import RandomForestClassifier
    from sklearn import tree
    from sklearn import neighbors
    from sklearn.ensemble import AdaBoostClassifier
    from sklearn.ensemble import VotingClassifier
    from sklearn.ensemble import BaggingClassifier
    from sklearn.model_selection import train_test_split
    
    #y=np.array(df1['Status'])
    
    

    X=df1
    #df1.head(5)
    xtrain, xtest, ytrain, ytest = train_test_split(X, y, test_size = 0.3, random_state=30)
    # Random forest Model: 
    model= RandomForestClassifier(n_estimators = 300,random_state =30,n_jobs=3,verbose=2,max_features=None)#,class_weight={0:100,1:10000,2:500})
    # Decion Tree Model: 
    decision_tree = tree.DecisionTreeClassifier(random_state=30, max_depth=100)
    # AdaBoost Classifier Model:
    clf = AdaBoostClassifier(base_estimator=model,n_estimators=100, random_state=0)
    # KNN :
    mo = neighbors.KNeighborsClassifier(n_neighbors =2)
    # Bagging Classifier Model:  
    bagging = BaggingClassifier(mo,max_samples=0.5, max_features=0.5)
    #Voting Classifier
    eclf = VotingClassifier(estimators=[ ('knn_bagging', bagging), ('adaboost', clf), ('RF',model)],voting='soft')
    eclf1 = eclf.fit(xtrain, ytrain)
    print('TRAINING DONE ......................')
    eclf1=eclf.predict(xtrain)
    eclf2=eclf.predict(xtest)
    print('TRAINING RESULTS-----------------------------------------')
    print("Accuracy: for Training :",metrics.accuracy_score(ytrain, eclf1)*100)
    print("Accuracy: for Testing :",metrics.accuracy_score(ytest, eclf2)*100)
    print()
    print('Confusion Matrix for Random Forest')
    print(metrics.confusion_matrix(eclf2, ytest))
    print()
    print('--------------------------------------------------------------------------------------------------------------------')
    return eclf

In [30]:
def predict(df1,clf,busa,ccar,month,zone,bran,payt,date):
    import numpy as np
    import pandas as pd
    import copy
    from sklearn import preprocessing
    
    
    df1['foo']= pd.to_datetime(df1['Net.Due.Dt'],format='%Y%m%d') - pd.to_datetime(date,format='%Y%m%d')

    df1['foo']=df1['foo'].dt.days
    df1['foo']=pd.to_numeric(df1['foo'])
    df1=df1[df1['foo']<6]
    df1.drop(columns='foo',inplace=True)
    df1.dropna(inplace=True)
    df1 = df1.reset_index()
    df1.drop(columns='index',inplace=True)
    
    df=copy.deepcopy(df1)
    print(df.shape)
    df['Prediction']=np.zeros(len(df1))
    print(df1.shape)
    df1.dropna(inplace=True)
    df1 = df1.reset_index()
    df1.drop(columns='index',inplace=True)
    #df1['Status']=df1['Status'].astype('category').cat.codes)
    #df1['Status']=df1['Status'].astype('category').cat.codes
    
    df1['PayT']= pd.to_datetime(df1['Net.Due.Dt'],format='%Y%m%d') - pd.to_datetime(df1['Pstng.Date'],format='%Y%m%d')
    df1['PayT']=df1['PayT'].dt.days
    df1['PayT']=pd.to_numeric(df1['PayT'])
    df1.drop(columns='Clearing',inplace=True)
    df1 = df1.reset_index()
    df1.drop(columns = 'index',inplace=True)
    #df1['Status']=df1['Status'].astype('category').cat.codes
    df1['BusA']=busa.transform(df1['BusA'])
    df1['CCAr']=ccar.transform(df1['CCAr'])#.astype('category').cat.codes
    #df1['Account']=df1['Account'].astype('category').cat.codes
    df1['Month']=month.transform(df1['Month'])#.astype('category').cat.codes
    df1.drop(columns='Reference',inplace=True)
    df1.drop(columns='Customer.Name',inplace=True)
    df1.drop(columns=['DocumentNo','Year'],inplace=True)
    df1['Zone']=zone.transform(df1['Zone'])#.astype('category').cat.codes
    df1['Bran']=bran.transform(df1['Bran'])#.astype('category').cat.codes
    #df1['PayT'] =payt.transform(df1['PayT'])#.astype('category').cat.codes
    df1.drop(columns='Doc.Chq.dt',inplace=True)
    df1.drop(columns='Ty',inplace=True)
    df1.drop(columns='Sale.Type',inplace=True)
    #df1['Pstng.Date']=pd.to_datetime(df1['Pstng.Date'],dayfirst=True).dt.strftime("%Y%m%d").astype(int)
    #df1['Net.Due.Dt']=pd.to_datetime(df1['Net.Due.Dt'],dayfirst=True).dt.strftime("%Y%m%d").astype(int)
    #df1['Clearing']=pd.to_datetime(df1['Clearing']).dt.strftime("%Y%m%d").astype(str)
    #df1.drop(columns=['Net.Due.Dt','Clearing'],inplace=True)
    #df1.drop(columns='Net.Due.Dt',inplace=True)
    df1 = df1.reset_index()
    df1.drop(columns='index',inplace=True)
    df1.drop(columns='Arr..Clearing...Net.Due.Date.',inplace=True)
    df1.drop(columns='G.L',inplace=True)
    
    #y1=df1.columns
    #x = df1.values #returns a numpy array
    #min_max_scaler = preprocessing.MinMaxScaler()
    #x_scaled = min_max_scaler.fit_transform(x)
    #df1 = pd.DataFrame(x_scaled,columns=df1.columns)
    
    #df1['Account']=df1['Account']/100000
    #df1['Reference']=df1['Reference']/1000000000
    #df1['Pstng.Date']=df1['Pstng.Date']/100000000
    #df1['Net.Due.Dt']=df1['Pstng.Date']/100000000
    
    
    print('Starting Prediction ....')
    prediction=clf.predict(df1)
    print('Generating Probabilities.....')
    predictprob=clf.predict_proba(df1)
    print('Finished prediction...')
    print()
    #df['Probability']=np.zeros(len(df))
    print(df1.head())
    

    arr=['No Delay','Low Delay','High Delay']
    arr1=['Low delay','Low Medium Delay','High Delay']#,'High Delay']#,'30 to 180','>180']#,'8-15 days','16-30 days','31-60 days','61-90 days','90-180 days','>180 days']
    for i in range(len(arr)):
        df[arr1[i]]=np.empty(len(df))
    print('Computing Prediction Table............')

    df['Prediction']=prediction
    #df['Prediction']=np.where(df['Prediction'] == 0, 'No Delay',(np.where(df['Prediction'] == 1, 'Low Delay',(np.where(df['Prediction'] == 2, 'High Delay','')))))
    for i in range(len(arr1)):
        df[arr1[i]]=np.around(predictprob[:,i],decimals=2)
        
    print(df.head(5))

    
    
    
        
    
    
    
    print(df.head())
    print('Writing to csv file')
    print('-----------------------------------------------------------------------------------------')
    df.to_csv('Predictedout.csv',index=False)

In [5]:
# import time
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import copy
trainpath='MyData.csv'

print('Reading data...')
df1=pd.read_csv(trainpath)
print('Finished Reading...')
print()
print(df1.head(5))
print()
date=20181101
df3=copy.deepcopy(df1)
print('Preprocessing started .......')
df2,df4,y,busa,ccar,month,zone,bran,payt=preprocessor(df1,date)
print(df2.head(5))
print('Preprocessing Done...........')
print('--------------------------------------------------------------------------------------------------------')
#print('RED ZONE VISUALISATION....')

#visualisation_red(df3)

Reading data...
Finished Reading...

   BusA CCAr     Reference  Account          Customer.Name   Zone  Bran  Ty  \
0  1000  BPR  4.132000e+09   929488      KAUSALYA AGENCIES  South  SBAD  RV   
1  9600  BPR  1.115502e+09   928860  TATA STEEL PROCESSING  North  GBAD  RV   
2  9600  BPR  1.115502e+09   928860  TATA STEEL PROCESSING  North  GBAD  RV   
3  1000  BPR  2.412166e+09   928860  TATA STEEL PROCESSING  North  GBAD  RV   
4  2500  BPR  2.410320e+09   928860  TATA STEEL PROCESSING  North  GBAD  RV   

    Status  Local.Crcy.Amt  ...  Doc.Chq.dt Month Pstng.Date  Net.Due.Dt  \
0   Not OD            0.10  ...  31.12.2018   Dec   31-12-18    05-01-19   
1  Overdue            0.04  ...  31.10.2018   Oct   31-10-18    10-11-18   
2   Not OD            0.12  ...  05.11.2018   Nov   05-11-18    15-11-18   
3  Overdue            0.17  ...  30.12.2018   Dec   30-12-18    09-01-19   
4  Overdue            0.16  ...  03.11.2018   Nov   03-11-18    13-11-18   

       G.L  Year Clrng.doc.  Cl

In [6]:
df4.drop(columns=['Status','Clrng.doc.'],inplace=True)

In [28]:
import time
t1=time.time()
print('Training Starting............')
clf=trainer(df2,y)
print('Training Done...................')
print('Time Taken',time.time()-t1)
print('-----------------------------------------------------------------------------------')

Training Starting............


[Parallel(n_jobs=3)]: Using backend ThreadingBackend with 3 concurrent workers.


building tree 1 of 300building tree 2 of 300

building tree 3 of 300
building tree 4 of 300
building tree 5 of 300
building tree 7 of 300
building tree 6 of 300
building tree 8 of 300
building tree 9 of 300
building tree 10 of 300
building tree 11 of 300
building tree 12 of 300
building tree 13 of 300building tree 14 of 300

building tree 15 of 300
building tree 16 of 300
building tree 17 of 300
building tree 18 of 300
building tree 19 of 300
building tree 20 of 300
building tree 21 of 300
building tree 22 of 300
building tree 23 of 300
building tree 24 of 300
building tree 25 of 300
building tree 26 of 300
building tree 27 of 300
building tree 28 of 300
building tree 29 of 300
building tree 30 of 300
building tree 31 of 300
building tree 32 of 300
building tree 33 of 300
building tree 34 of 300
building tree 35 of 300
building tree 36 of 300
building tree 37 of 300
building tree 38 of 300


[Parallel(n_jobs=3)]: Done  35 tasks      | elapsed:    7.9s


building tree 39 of 300
building tree 40 of 300
building tree 41 of 300building tree 42 of 300

building tree 43 of 300
building tree 44 of 300
building tree 45 of 300
building tree 46 of 300
building tree 47 of 300
building tree 48 of 300
building tree 49 of 300
building tree 50 of 300
building tree 51 of 300
building tree 52 of 300
building tree 53 of 300
building tree 54 of 300
building tree 55 of 300
building tree 56 of 300
building tree 57 of 300
building tree 58 of 300
building tree 59 of 300
building tree 60 of 300
building tree 61 of 300
building tree 62 of 300
building tree 63 of 300
building tree 64 of 300
building tree 65 of 300
building tree 66 of 300
building tree 67 of 300
building tree 68 of 300
building tree 69 of 300
building tree 70 of 300
building tree 71 of 300
building tree 72 of 300
building tree 73 of 300
building tree 74 of 300
building tree 75 of 300
building tree 76 of 300
building tree 77 of 300building tree 78 of 300

building tree 79 of 300
building tree 80

...

[Parallel(n_jobs=3)]: Done 300 out of 300 | elapsed:  1.0min finished
[Parallel(n_jobs=3)]: Using backend ThreadingBackend with 3 concurrent workers.


TRAINING DONE ......................


[Parallel(n_jobs=3)]: Done  35 tasks      | elapsed:    0.5s
[Parallel(n_jobs=3)]: Done 156 tasks      | elapsed:    2.2s
[Parallel(n_jobs=3)]: Done 300 out of 300 | elapsed:    4.3s finished
[Parallel(n_jobs=3)]: Using backend ThreadingBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  35 tasks      | elapsed:    0.3s
[Parallel(n_jobs=3)]: Done 156 tasks      | elapsed:    1.6s


TRAINING RESULTS-----------------------------------------
Accuracy: for Training : 99.13159876181712
Accuracy: for Testing : 94.90122589209027

Confusion Matrix for Random Forest
[[19686   528   195]
 [  308  2478    87]
 [  126    62  2144]]

--------------------------------------------------------------------------------------------------------------------
Training Done...................
Time Taken 69.89135527610779
-----------------------------------------------------------------------------------


[Parallel(n_jobs=3)]: Done 300 out of 300 | elapsed:    3.1s finished


In [8]:
print('Feature Importances')
for feature in zip(df2.columns, clf.feature_importances_):
    print(feature)

Feature Importances
('BusA', 0.021979008024188045)
('CCAr', 0.0)
('Account', 0.10384891254705847)
('Zone', 0.020164515969886904)
('Bran', 0.06386870879101042)
('Local.Crcy.Amt', 0.06235064220318058)
('PayT', 0.0446286290726739)
('Month', 0.014991299319824526)
('Pstng.Date', 0.13992426633514315)
('Net.Due.Dt', 0.3106330067652006)
('Pending_Amy', 0.15454106617222488)
('pending_cnt', 0.06306994479960856)


In [31]:
testpath= ''
#test=pd.read_csv(testpath)
predict(df4,clf,busa,ccar,month,zone,bran,payt,date)
print() 
print('______________________________________  END  ____________________________________')
print()
print()

(5699, 22)
(5699, 22)
Starting Prediction ....


[Parallel(n_jobs=3)]: Using backend ThreadingBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  35 tasks      | elapsed:    0.0s
[Parallel(n_jobs=3)]: Done 156 tasks      | elapsed:    0.2s
[Parallel(n_jobs=3)]: Done 300 out of 300 | elapsed:    0.4s finished
[Parallel(n_jobs=3)]: Using backend ThreadingBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  35 tasks      | elapsed:    0.0s


Generating Probabilities.....


[Parallel(n_jobs=3)]: Done 156 tasks      | elapsed:    0.2s
[Parallel(n_jobs=3)]: Done 300 out of 300 | elapsed:    0.5s finished


Finished prediction...

   BusA  CCAr  Account  Zone  Bran  Local.Crcy.Amt  PayT  Month  Pstng.Date  \
0     0     0   928860     1     9            0.14     0      5    20181011   
1     6     0   928860     1     9            0.13     0      5    20181010   
2     6     0   927881     2    21            0.17     0      2    20180712   
3     6     0   927881     2    21            0.16     0      3    20180612   
4     6     0   927881     2    21            0.13     0      3    20180612   

   Net.Due.Dt  Pending_Amy  pending_cnt  
0    20181011         0.14            1  
1    20181010         0.14            1  
2    20180712         0.00            0  
3    20180612         0.00            0  
4    20180612         0.00            0  
Computing Prediction Table............
   BusA CCAr     Reference  Account          Customer.Name   Zone  Bran  Ty  \
0  1000  BPR  2.452146e+09   928860  TATA STEEL PROCESSING  North  GBAD  RV   
1  9600  BPR  2.452146e+09   928860  TATA STEEL PROC